In [1]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
import pandas as pd
import unidecode
import numpy as np
import operator
from operator import itemgetter
from itertools import groupby
import re

In [3]:
# #demo
# str = "Xin chào, tôi tên là Tùng và đây là bản demo của tôi.\nIn English, this's Tung."
# new_str = unidecode.unidecode(str)
# print(new_str)

# **Đọc dữ liệu**

In [4]:
agoda_path = "/content/drive/MyDrive/20212/THDL/data/new_agoda_normalized.csv"
booking_path = "/content/drive/MyDrive/20212/THDL/data/new_booking_normalized.csv"
mytour_path = "/content/drive/MyDrive/20212/THDL/data/mytour_normalized.csv"
traveloka_path = "/content/drive/MyDrive/20212/THDL/data/traveloka_normalized.csv"

In [5]:
agoda_df = pd.read_csv(agoda_path)
booking_df = pd.read_csv(booking_path, lineterminator='\n')
mytour_df = pd.read_csv(mytour_path)
traveloka_df = pd.read_csv(traveloka_path)

In [6]:
agoda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           807 non-null    object 
 1   hotel name     807 non-null    object 
 2   url            807 non-null    object 
 3   address        807 non-null    object 
 4   stars          807 non-null    float64
 5   price          807 non-null    int64  
 6   rating         807 non-null    float64
 7   n_reviews      807 non-null    int64  
 8   image          807 non-null    object 
 9   reviews        807 non-null    object 
 10  facilities     807 non-null    object 
 11  nearby places  807 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 75.8+ KB


In [7]:
agoda_df.head(5)

,city,hotel name,url,address,stars,price,rating,n_reviews,image,reviews,facilities,nearby places
0,Nha Trang,Căn Hộ Star Beach (Star Beach Apartment),https://www.agoda.com/vi-vn/star-beach-nha-tra...,"02 Nguyen Thi Minh Khai Street, Lộc Thọ, Nha T...",5.0,445886,9.6,127,https://pix6.agoda.net/hotelImages/134/1348629...,"['Địa điểm đẹp và khách sạn đẹp, rộng rãi, vie...","['Dọn phòng hằng ngày', 'Điều hòa', 'Đưa đón s...","['Hon Chong - 3,9 km', 'Bảo tàng Hải dương học..."
1,Nha Trang,Khách Sạn Le's Cham (Le's Cham Hotel),https://www.agoda.com/vi-vn/le-s-cham-hotel_2/...,"87 Bạch Đằng, Tân Lập, Nha Trang, Việt Nam",4.0,370310,9.1,344,https://pix6.agoda.net/hotelImages/6845017/-1/...,"['Với trí tốt gần địa điểm ăn uống , đi biển g...","['Dọn phòng hằng ngày', 'Quán bar', 'Đưa đón s...","['Hon Chong - 3,98 km', 'i resort - 3,99 km', ..."
2,Nha Trang,LeMore Hotel Nha Trang,https://www.agoda.com/vi-vn/le-more-hotel_2/ho...,"33A Tô Hiến Thành, Phường Tân Lập, Tân Lập, Nh...",4.0,436506,9.0,899,https://pix6.agoda.net/hotelImages/6411523/-1/...,[],"['Đưa đón sân bay', 'Wi-Fi miễn phí trong tất ...","['Hon Chong - 4,03 km', 'i resort - 4,06 km', ..."
3,Nha Trang,Căn hộ Kymodo Panorama (Kymodo Panorama Apartm...,https://www.agoda.com/vi-vn/kymodo-apartment/h...,"02 Nguyen Thi Minh Khai Street, Panorama Build...",5.0,524571,9.3,267,https://pix5.agoda.net/hotelImages/10780056/-1...,['Gia đình đã có một kỳ nghỉ tuyệt vời tại đây...,"['Nằm ở trung tâm Nha Trang', 'Điều hòa', 'Đưa...","['Hon Chong - 3,97 km', 'Bảo tàng Hải dương họ..."
4,Nha Trang,InterContinental Nha Trang,https://www.agoda.com/vi-vn/intercontinental-n...,"32-34 Tran Phu Street, Lộc Thọ, Nha Trang, Việ...",5.0,2538532,9.1,1966,https://pix5.agoda.net/hotelImages/564/564356/...,"['(+)Khách sạn đẹp, nhân viên thân thiện và nh...","['Dọn phòng hằng ngày', 'Quán bar', 'Đưa đón s...","['Hon Chong - 3,3 km', 'i resort - 3,76 km', '..."


In [8]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4845 entries, 0 to 4844
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               4845 non-null   object 
 1   Hotel name         4845 non-null   object 
 2   Url                4845 non-null   object 
 3   Address            4845 non-null   object 
 4   Stars              4845 non-null   int64  
 5   Price              4845 non-null   int64  
 6   Rating             4845 non-null   float64
 7   Number of reviews  4845 non-null   int64  
 8   Reviews            4845 non-null   object 
 9   Facilities         4845 non-null   object 
 10  Description        4845 non-null   object 
 11  Nearby places      4845 non-null   object 
 12  Image              4845 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 492.2+ KB


In [9]:
booking_df.head(5)

,City,Hotel name,Url,Address,Stars,Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places,Image
0,Hà Nội,Khách sạn HANZ Regal Hotel Hanoi,https://www.booking.com/hotel/vn/90-pham-huy-t...,"90 Pham Huy Thong, Quận Ba Đình, Hà Nội, Viê...",2,421890,7.8,86,"['“máy lạnh mát, wifi trong từng phòng nên mạn...","['Xe đưa đón sân bay', 'Wi-Fi miễn phí', 'P...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Hồ Ngọc Khánh', '0,3 km'), ('Trung tâm thươ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
1,Hà Nội,Khách sạn La Nueva Boutique Hotel Hanoi & Spa,https://www.booking.com/hotel/vn/hanoi-eleganc...,"32 Lo Su Street, Quận Hoàn Kiếm, Hà Nội, Viê...",4,1316000,9.3,209,"['“Địa điểm tuyệt vời, đi một xíu là ra đến hồ...","['Xe đưa đón sân bay', 'Phòng không hút thu...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Nhà hát múa rối Thăng Long', '0,1 km'), ('Đ...",['https://cf.bstatic.com/xdata/images/hotel/ma...
2,Hà Nội,Khách sạn La Storia Ruby Hotel,https://www.booking.com/hotel/vn/hanoi-eleganc...,"3 Yen Thai Street, Quận Hoàn Kiếm, Hà Nội, V...",3,625000,9.6,217,['“- vị trí gần hồ gươm rất tiện cho việc đi b...,"['Xe đưa đón sân bay', 'Phòng không hút thu...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Quảng Trường Đông Kinh Nghĩa Thục', '0,4 km...",['https://cf.bstatic.com/xdata/images/hotel/ma...
3,Hà Nội,Khách sạn LeaH Silk Hotel,https://www.booking.com/hotel/vn/leah-silk.vi....,"41- 43 Hàng Bông, Hoàn Kiếm , Hà Nội, Quận Hoà...",4,1800000,9.1,72,"['“Mình ở phòng có cửa thông với phòng khác, v...","['1 hồ bơi', 'Xe đưa đón sân bay', 'Phòng kh...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Nhà thờ Thánh Joseph', '0,3 km'), ('Quảng T...",['https://cf.bstatic.com/xdata/images/hotel/ma...
4,Hà Nội,Khách sạn Babylon Premium Hotel & Spa,https://www.booking.com/hotel/vn/babylon-premi...,"05-07 Hàng Hòm, Quận Hoàn Kiếm, Hà Nội, Việ...",4,1262250,9.4,627,['“Phòng rất rộng và sạch.\nVị trí ngay giữa k...,"['Xe đưa đón sân bay', 'Phòng không hút thu...",\r\nBạn đủ điều kiện nhận giảm giá Genius tại ...,"[('Quảng Trường Đông Kinh Nghĩa Thục', '0,3 km...",['https://cf.bstatic.com/xdata/images/hotel/ma...


In [10]:
traveloka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 0 to 1058
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1059 non-null   object 
 1   Hotel name         1059 non-null   object 
 2   Image              1059 non-null   object 
 3   Url                1059 non-null   object 
 4   Address            1059 non-null   object 
 5   Stars              1059 non-null   int64  
 6   Min Price          1059 non-null   int64  
 7   Rating             1059 non-null   float64
 8   Number of reviews  1059 non-null   int64  
 9   Reviews            1059 non-null   object 
 10  Facilities         1059 non-null   object 
 11  Description        1059 non-null   object 
 12  Nearby places      1059 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 107.7+ KB


In [11]:
traveloka_df.head(5)

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
0,Hà Nội,Khách sạn MK Premier Boutique,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"72-74 Hàng Buồm, Hàng Buồm, Quận Hoàn Kiếm,...",4,616173,8.7,113,"['khách sạn ở hành lang không có điều hòa, đườ...","['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Đền Bạch Mã', '12 m'), ('Hanoi Train Street..."
1,Hà Nội,Classy Holiday Hotel & Spa,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"49 Lãn Ông, Hàng Bồ, Quận Hoàn Kiếm, Hà Nội, ...",4,487800,9.4,177,['Phục vụ dọn phòng mở cửa phòng không gọi cửa...,"['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('King Pirates Pub', '229 m'), ('Hanoi Train ..."
2,Hà Nội,La Beaute Boutique Hotel & Spa,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"11 Ngõ Trung Yên, phố Đinh Liệt, Hoan Kiếm, Hà...",4,600000,8.1,28,"['phòng nhỏ, xe ô tô không đi vào được nhưng p...","['Máy lạnh', 'Nhà hàng', 'Lễ tân 24h', 'Thang ...",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Nhà hát Ca trù Thăng Long', '94 m'), ('Hano..."
3,Hà Nội,Căn Hộ Wecozy,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"Số 20, Tống Duy Tân, Phố Hàng Bống, Hàng Bông,...",0,741572,9.0,60,"['', 'Không gian căn hộ rộng rãi, thoáng mát, ...","['Máy lạnh', 'Thang máy', 'WiFi']",Giờ nhận và trả phòng\r\nGiờ nhận phòng: 15:00...,"[('Hanoi Train Street', '106 m'), ('Nhà hát Tu..."
4,Hà Nội,Green Hotel & Apartment HN,['https://ik.imagekit.io/tvlk/apr-asset/dgXfoy...,https://www.traveloka.com/vi-vn/hotel/vietnam/...,"No 5, alley 148 Tran Duy Hung street, Yen Hoa ...",3,369750,7.9,42,['Chất lượng quá kém. Làm dịch vụ phòng ở khôn...,"['Máy lạnh', 'Lễ tân 24h', 'Thang máy']",Giờ nhận và trả phòng\r\nGiờ nhận phòng: Từ 14...,"[('Chùa Cảm Ứng', '694 m'), ('Đại học Quốc gia..."


In [12]:
mytour_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1034 non-null   object 
 1   Hotel name         1034 non-null   object 
 2   Image              1034 non-null   object 
 3   Url                1034 non-null   object 
 4   Address            1034 non-null   object 
 5   Stars              1034 non-null   int64  
 6   Min Price          1034 non-null   int64  
 7   Rating             1034 non-null   float64
 8   Number of reviews  1034 non-null   int64  
 9   Reviews            1034 non-null   object 
 10  Facilities         1034 non-null   object 
 11  Description        1034 non-null   object 
 12  Nearby places      1034 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 105.1+ KB


In [13]:
mytour_df.head(5)

,City,Hotel name,Image,Url,Address,Stars,Min Price,Rating,Number of reviews,Reviews,Facilities,Description,Nearby places
0,Hà Nội,Khách Sạn Apricot Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/954-khach-san-apri...,"136, Hàng Trống, Quận Hoàn Kiếm, Hà Nội, Việt Nam",5,1908235,9.2,320,['Nhận xét chất lượng phòng\nNV lịch sự. Chất ...,"['American Express', 'Lễ tân 24h', 'Bán tour',...",['Khách sạn Apricot Hotel mang vẻ đẹp lỗng lẫy...,"[('Kanko Izakaya', '60 m'), ('Nhà hàng Lục Thủ..."
1,Hà Nội,Khách Sạn Imperial Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/972-khach-san-impe...,"44, Hàng Hành, Quận Hoàn Kiếm, Hà Nội, Việt Nam",4,852946,9.4,159,['Đánh giá trải nghiệm khách sạn\nKhách sạn đẹ...,"['Lễ tân 24h', 'Trợ giúp đặc biệt', 'Giữ hành ...",['Imperial Hotel Hanoi đi và hoạt động từ năm ...,"[('Nhà hàng và cafe Old Street', '25 m'), ('Po..."
2,Hà Nội,Khách Sạn Beryl Palace Hanoi,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/49198-khach-san-be...,"173a, Hàng Bông, Quận Hoàn Kiếm, Hà Nội, Việt Nam",4,638138,9.2,42,['Trải nghiệm tuyệt vời!\nMình đi nghỉ với gia...,"['Đưa/đón khách sân bay', 'Cho thuê xe đạp', '...",['Lối trang trí kiểu cổ điển của Pháp và kiến ...,"[('O Lounge, Bistro and Sport Bar', '9 m'), ('..."
3,Hà Nội,Khách sạn La Nueva Boutique Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=840...,https://mytour.vn/khach-san/1486-khach-san-la-...,"32, Lò Sũ, Quận Hoàn Kiếm, Hà Nội, Việt Nam",4,702426,9.4,185,['nhận xét về khách sạn và sự phản hồi chậm củ...,"['Thu đổi ngoại tệ', 'Đặt vé xe/máy bay', 'Giữ...","['CHÀO MỪNG BẠN ĐẾN VỚI', 'LA NUEVA BOUTIQUE H...","[('Nhà Hàng The Crown', '7 m'), ('The Gourmet ..."
4,Hà Nội,Khách Sạn Melia Hà Nội,['https://img.tripi.vn/cdn-cgi/image/width=124...,https://mytour.vn/khach-san/938-khach-san-meli...,"44b, Lý Thường Kiệt, Quận Hoàn Kiếm, Hà Nội, V...",5,2242177,8.8,322,['Tốt\nPhòng Deluxe Room cung cấp cũng khá ổn ...,"['Bán tour', 'Thu đổi ngoại tệ', 'Lễ tân 24h',...",['Khách Sạn Melia Hà Nội sở hữu một vị trí đắc...,"[('Aha Cafe', '80 m'), ('Moonshine Cafeteria &..."


# **Tiền xử lý dữ liệu cho String matching**

**Hotel name**: 


*   Loại bỏ các từ con hay gặp: Hotel, Khách sạn,...
*   Đưa về dạng chữ thường không dấu

*   Nếu có 2 tên tiếng anh và tiếng việt thì chỉ giữ lại 1 tên





In [14]:
#Bước 1: Chuyển tên KS về dạng chữ thường
traveloka_df['new_name'] = traveloka_df['Hotel name'].str.lower()
mytour_df['new_name'] = mytour_df['Hotel name'].str.lower()
agoda_df['new_name'] = agoda_df['hotel name'].str.lower()
booking_df['new_name'] = booking_df['Hotel name'].str.lower()

In [15]:
booking_df['new_name']

0                        khách sạn hanz regal hotel hanoi
1           khách sạn la nueva boutique hotel hanoi & spa
2                          khách sạn la storia ruby hotel
3                               khách sạn leah silk hotel
4                   khách sạn babylon premium hotel & spa
                              ...                        
4840          nhà khách halong overnight in cat ba island
4841    nhà khách room in boat - halong bay overnight ...
4842                       căn hộ mới coral villa ha long
4843    du thuyền legend halong private cruises - mana...
4844    nhà khách room in boat - halong bay cruise for...
Name: new_name, Length: 4845, dtype: object

In [16]:
#Bước 2: Loại bỏ những substring thông dụng: khách sạn, căn hộ,...
subnames = ["căn hộ", "apartment", 
            "khách sạn", "hotel", "hostel", "nhà khách", "nhà thuyền"
            "khu", "nghỉ dưỡng", "nghỉ mát", "resort", "spa",
            "nhà", "nhà dân dịch vụ", "home", "house", "homestay", "serviced homestay",
            "biệt thự", "villa", 
            "du thuyền", "cruise", 
            "gần", "sang trọng", "near", "luxury", "mới", 
            "bungalow", "restaurant", "residences", "studio", "bar", 
            "và", "and", "&"
            ]
agoda_df['new_name'].replace(subnames,'', inplace=True, regex=True)
booking_df['new_name'].replace(subnames,'', inplace=True, regex=True)
traveloka_df['new_name'].replace(subnames,'', inplace=True, regex=True)
mytour_df['new_name'].replace(subnames,'', inplace=True, regex=True)

In [17]:
booking_df['new_name']

0                                  hanz regal  hanoi
1                         la nueva boutique  hanoi  
2                                    la storia ruby 
3                                         leah silk 
4                                 babylon premium   
                            ...                     
4840                  halong overnight in cat ba isl
4841            room in boat - halong bay overnight 
4842                                  coral  ha long
4843     legend halong private s - managed by bhaya 
4844      room in boat - halong bay  for backpackers
Name: new_name, Length: 4845, dtype: object

In [18]:
#Bước 3: Bỏ dấu
agoda_df['new_name']=[unidecode.unidecode(y) for y in agoda_df['new_name']]
booking_df['new_name']=[unidecode.unidecode(y) for y in booking_df['new_name']]
traveloka_df['new_name']=[unidecode.unidecode(y) for y in traveloka_df['new_name']]
mytour_df['new_name']=[unidecode.unidecode(y) for y in mytour_df['new_name']]

In [19]:
booking_df['new_name'].head(10)

0                           hanz regal  hanoi
1                  la nueva boutique  hanoi  
2                             la storia ruby 
3                                  leah silk 
4                          babylon premium   
5      sumitomo   service  20 lane 12 dao tan
6                         bella rosa suite   
7                           the noble swan   
8                             hanoi    travel
9                           old quarter   gym
Name: new_name, dtype: object

In [20]:
#Bước 4: Với những tên KS có 2 phần tiếng anh, tiếng việt --> lấy 1 phần
agoda_df['new_name'] = [y.partition('(')[0] for y in agoda_df['new_name']]
booking_df['new_name'] = [y.partition('(')[0] for y in booking_df['new_name']]
traveloka_df['new_name'] = [y.partition('(')[0] for y in traveloka_df['new_name']]
mytour_df['new_name'] = [y.partition('(')[0] for y in mytour_df['new_name']]

In [21]:
mytour_df['new_name'].head(5)

0               apricot ha noi
1              imperial ha noi
2           beryl palace hanoi
3     la nueva boutique ha noi
4                 melia ha noi
Name: new_name, dtype: object

In [22]:
#Bước 5: Loại bỏ khoảng cách giữa các từ
agoda_df['new_name'].replace(' ','', inplace=True, regex=True)
booking_df['new_name'].replace(' ','', inplace=True, regex=True)
traveloka_df['new_name'].replace(' ','', inplace=True, regex=True)
mytour_df['new_name'].replace(' ','', inplace=True, regex=True)

In [23]:
traveloka_df['new_name'].head(5)

0    mkpremierboutique
1        classyholiday
2     labeauteboutique
3               wecozy
4              greenhn
Name: new_name, dtype: object

In [24]:
#Check kết quả
for i in range(100):
    print(traveloka_df['Hotel name'][i]+' ==> '+traveloka_df['new_name'][i])
print("\n====================\n")
for i in range(100):
    print(booking_df['Hotel name'][i]+' ==> '+booking_df['new_name'][i])
print("\n====================\n")
for i in range(100):
    print(agoda_df['hotel name'][i]+' ==> '+agoda_df['new_name'][i])
print("\n====================\n")
for i in range(100):
    print(mytour_df['Hotel name'][i]+' ==> '+mytour_df['new_name'][i])

Khách sạn MK Premier Boutique ==> mkpremierboutique
Classy Holiday Hotel & Spa ==> classyholiday
La Beaute Boutique Hotel & Spa ==> labeauteboutique
Căn Hộ Wecozy ==> wecozy
Green Hotel & Apartment HN ==> greenhn
Chariot Hotel ==> chariot
Khách sạn Grand Plaza Hà Nội ==> grplazahanoi
VNAHOMES APARTHOTEL ==> vnasapart
Lucien Hanoi Hotel & Spa ==> lucienhanoi
SOJO Ga Hanoi ==> sojogahanoi
Hanoi Le Jardin Hotel & Spa ==> hanoilejardin
Mina Hotel & Spa ==> mina
Khách Sạn Halais ==> halais
Hanoi E Central Hotel ==> hanoiecentral
Hanoi Emerald Waters Hotel & Spa ==> hanoiemeraldwaters
Khách sạn Bảo Sơn ==> baoson
Romantique Hotel De Hanoi ==> romantiquedehanoi
Millennium Hanoi Hotel ==> millenniumhanoi
ISTAY Hotel Apartment 6 ==> istay6
Honeymoon Hotel and Apartment ==> honeymoon
New Style House Hotel ==> newstyle
V-Studio Hotel Apartment 3 ==> v-3
Hanoi Brilliant Hotel and Spa ==> hanoibrilliant
My Hotel 23 ==> my23
Little Hanoi Hotel ==> littlehanoi
Spring Hotel Hanoi ==> springhanoi
Khách

In [25]:
# demodf = agoda_df.copy(deep=True)
# demodf['new_name'].head(10)

**Address**: 


*   Loại bỏ các từ con hay gặp: đường, quận,...
*   Đưa về dạng chữ thường không dấu

In [26]:
#Bước 1: Chuyển về dạng chữ thường
#Bước 1: Chuyển tên KS về dạng chữ thường
traveloka_df['new_add'] = traveloka_df['Address'].str.lower()
mytour_df['new_add'] = mytour_df['Address'].str.lower()
agoda_df['new_add'] = agoda_df['address'].str.lower()
booking_df['new_add'] = booking_df['Address'].str.lower()

In [27]:
traveloka_df['new_add'].head(5)

0    72-74 hàng buồm, hàng buồm, quận hoàn kiếm,...
1    49 lãn ông, hàng bồ, quận hoàn kiếm, hà nội, ...
2    11 ngõ trung yên, phố đinh liệt, hoan kiếm, hà...
3    số 20, tống duy tân, phố hàng bống, hàng bông,...
4    no 5, alley 148 tran duy hung street, yen hoa ...
Name: new_add, dtype: object

In [28]:
#Kiểm tra xem có bản ghi nào chứa "vn" hay không?
m1_df = agoda_df[agoda_df['new_add'].str.contains('vn')]
m1_df

,city,hotel name,url,address,stars,price,rating,n_reviews,image,reviews,facilities,nearby places,new_name,new_add
619,Hà Nội,Khách sạn Ancient Lane (Ancient Lane Hotel),https://www.agoda.com/vi-vn/ancient-lane-hotel...,"Số 2 đường Tam Thương, Phường Hàng Gai, Quận H...",35.0,662338,9.0,1236,https://pix6.agoda.net/hotelImages/5100415/-1/...,['địa điểm thì do nằm trong hẻm vì Hà nội đặt ...,"['Cách phương tiện công cộng 950 m', 'Nằm ở tr...","['Việt Nam Bảo tàng Lịch sử quân sự - 800 m', ...",ancientlane,"số 2 đường tam thương, phường hàng gai, quận h..."


In [29]:
m2_df = booking_df[booking_df['new_add'].str.contains('vn')]
m3_df = traveloka_df[traveloka_df['new_add'].str.contains('vn')]
m4_df = mytour_df[mytour_df['new_add'].str.contains('vn')]

In [30]:
pd.set_option('max_rows', 999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')
m1_df['new_add']

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


619    số 2 đường tam thương, phường hàng gai, quận hoàn kiếm, thành phố hà nội, vn, phố cổ, hà nội, việt nam, 10000
Name: new_add, dtype: object

In [31]:
m2_df['new_add']

3837                                 căn hộ vnc - ct3 phước hải, nha trang, việt nam
3908               vinpearl vn-22 grand world phú quốc, ganh dau, phú quốc, việt nam
3923              gành dầu - cửa cạn vn-10 grand world, ganh dau, phú quốc, việt nam
3955    vn44 grand world, bãi dài, gành dầu, phú quốc., ganh dau, phú quốc, việt nam
4143           grand world, gành dầu, phú quốc vn - 03, ganh dau, phú quốc, việt nam
Name: new_add, dtype: object

In [32]:
m3_df['new_add']

Series([], Name: new_add, dtype: object)

In [33]:
m4_df['new_add']

Series([], Name: new_add, dtype: object)

Nhận thấy, 3 bộ dữ liệu Booking, Mytour, Traveloka không có cụm viết tắt "vn" chỉ Việt Nam. Bộ Agoda tuy có nhưng không quá ảnh hưởng.

In [34]:
# #demo
# demo1 = "số 2 đường tam thương, phường hàng gai, quận hoàn kiếm, thành phố hà nội, vn, phố cổ, hà nội, việt nam, 10000"  #demo 
# demo_a = "270 Võ Nguyên Giáp, Mỹ An, Ngũ Hành Sơn, Đà Nẵng, Đà Nẵng, Việt Nam 51 Trần Bạch Đằng, Mỹ An, Ngũ Hành Sơn, Đà Nẵng, Việt Nam, Đà Nẵng, Việt Nam, 70000" #demo
# demo2 = "06 luong ngoc quyen str hoan kiem dist, quận hoàn kiếm, hà nội, việt nam"    #booking
# demo3 = "39 ton that thiep street, ben nghe ward (gan 1234 cho dong xuan_demo nhe), district 1, quận 1, tp. hồ chí minh, việt nam"   #booking
# demo4 = "89 - 91 vo van kiet st, phuoc my ward, son tra district, da nang city , viet nam, phước mỹ, dà nẵng, việt nam, 550000"  #agoda
# demo5 = "117 trần duy hưng, phường trung hòa, quận cầu giấy, hà nội, việt nam, 100000" #traveloka
# demo6 = "20-22, chân cầm, quận hoàn kiếm, hà nội, việt nam"  #mytour
# demo7 = "96 Yên Thế, Phường 2, Quận Tân Bình, Thành phố Hồ Chí Minh, việt nam, Quận Tân Bình, Hồ Chí Minh, việt nam, 700000" #agoda

In [35]:
# demo_a = demo_a.lower()
# demo_a

In [36]:
# #for booking
# mm = demo_a.partition(', việt nam')[0]
# mm

In [37]:
# mm1 = demo7.partition(', việt nam')
# mm1

In [38]:
#Bước 2: Loại bỏ mã vùng/mã tỉnh 
traveloka_df['new_add'] = [y.partition(', việt nam')[0] for y in traveloka_df['new_add']]
booking_df['new_add'] = [y.partition(', việt nam')[0] for y in booking_df['new_add']]
agoda_df['new_add'] = [y.partition(', việt nam')[0] for y in agoda_df['new_add']]
mytour_df['new_add'] = [y.partition(', việt nam')[0] for y in mytour_df['new_add']]

In [39]:
traveloka_df['new_add'].head(10)

0                                                                     72-74 hàng buồm, hàng buồm, quận hoàn kiếm, hà nội
1                                                                              49 lãn ông, hàng bồ, quận hoàn kiếm, hà nội
2                                             11 ngõ trung yên, phố đinh liệt, hoan kiếm, hàng bạc, quận hoàn kiếm, hà nội
3                                                    số 20, tống duy tân, phố hàng bống, hàng bông, quận hoàn kiếm, hà nội
4    no 5, alley 148 tran duy hung street, yen hoa ward, cau giay district, hanoi, phường trung hòa, quận cầu giấy, hà nội
5                                                                        235 phố huế, phố huế, quận hai bà trưng, hà nội
6                                                               117 trần duy hưng, phường trung hòa, quận cầu giấy, hà nội
7                                                                                    30 phố võng thị , bưởi, tây hồ, hà nội
8       

In [40]:
agoda_df['new_add'].head(5)

0                       02 nguyen thi minh khai street, lộc thọ, nha trang
1                                         87 bạch đằng, tân lập, nha trang
2                    33a tô hiến thành, phường tân lập, tân lập, nha trang
3    02 nguyen thi minh khai street, panorama building, lộc thọ, nha trang
4                                32-34 tran phu street, lộc thọ, nha trang
Name: new_add, dtype: object

In [41]:
booking_df['new_add'].head(5)

0                         90 pham huy thong, quận ba đình, hà nội
1                         32 lo su street, quận hoàn kiếm, hà nội
2                       3 yen thai street, quận hoàn kiếm, hà nội
3    41- 43 hàng bông, hoàn kiếm , hà nội, quận hoàn kiếm, hà nội
4                          05-07 hàng hòm, quận hoàn kiếm, hà nội
Name: new_add, dtype: object

In [42]:
#Bước 3: Loại bỏ các substring thông dụng
subadds = ["xóm", "quận", "thị xã", "ngõ", "số", "tòa", "thành phố", "phố cổ", "phố",  
           "đường", "phường", "cư xá", "cảng", "hẻm", "khu", "đô thị", "nhà",
           "ngách", "phòng", "căn", "tầng", "tỉnh", "tổ", "ap", "ấp", "lô", "bãi",
           "xã", "huyện", "du lịch", "di tích", "đại lộ", "đảo", "và", "tel 02633.608084",
           "trung tâm", "đối diện", "phía", "sau", "trước", "siêu thị", "vn", "quan",
           "no", "no.", "nos", "island", "alley", "lane", "street", "ward", "district",
           "group", "province", "block", "beach", "city", "basilica", "market", "lake",
           "area", "unit", "gate", "port", "wharft", "near by", "building", "zone",
           "tp", "str", "st", "dist", "pro", "bis", "kđt", "hcmc", "hcm", 
           "hà nội", "hồ chí minh", "đà lạt", "đà nẵng", "nha trang", "vũng tàu", 
           "phú quốc", "quảng ninh", "hạ long", "ha long", "ha noi", "ho chi minh",
           "da lat", "da nang", "vung tau", "phu quoc", "quang ninh", "viet nam", "vietnam", 
           ","
            ]

In [43]:
# demodf = booking_df.copy(deep=True)
# demodf['new_add'].head(10)

In [44]:
# demodf['new_add'].replace(subadds,'', inplace=True, regex=True)

In [45]:
# demodf['new_add']

In [46]:
# demodf['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')

In [47]:
agoda_df['new_add'].replace(subadds,'', inplace=True, regex=True)
booking_df['new_add'].replace(subadds,'', inplace=True, regex=True)
traveloka_df['new_add'].replace(subadds,'', inplace=True, regex=True)
mytour_df['new_add'].replace(subadds,'', inplace=True, regex=True)

In [48]:
traveloka_df['new_add'].head(10)

0                                72-74 hàng buồm hàng buồm quận hoàn kiếm 
1                                         49 lãn ông hàng bồ quận hoàn kiếm 
2                11  trung yên  đinh liệt hoan kiếm hàng bạc quận hoàn kiếm 
3                       20 tống duy tân  hàng bống hàng bông quận hoàn kiếm 
4     5  148 tran duy hung  yen hoa  cau giay  hai  trung hòa quận cầu giấy 
5                                      235 phố huế  huế quận hai bà trưng 
6                                117 trần duy hưng  trung hòa quận cầu giấy 
7                                                  30  võng thị  bưởi tây hồ 
8                             2  cầu gỗ  hoàn kiếm  hàng bạc quận hoàn kiếm 
9                                 115 trần hưng đạo cửa  nam quận hoàn kiếm 
Name: new_add, dtype: object

In [49]:
# #Bước 4: Bỏ dấu
agoda_df['new_add']=[unidecode.unidecode(y) for y in agoda_df['new_add']]
booking_df['new_add']=[unidecode.unidecode(y) for y in booking_df['new_add']]
traveloka_df['new_add']=[unidecode.unidecode(y) for y in traveloka_df['new_add']]
mytour_df['new_add']=[unidecode.unidecode(y) for y in mytour_df['new_add']]

In [50]:
booking_df['new_add'].head(5)

0                 90 pham huy thong  ba dinh ha noi
1                       32 lo su   hoan kiem ha noi
2                     3 yen thai   hoan kiem ha noi
3    41- 43 hang bong hoan kiem    hoan kiem ha noi
4                  05-07 hang hom  hoan kiem ha noi
Name: new_add, dtype: object

In [51]:
#Bước 5: Loại bỏ khoảng cách giữa các từ
# agoda_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
# booking_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
# traveloka_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
# mytour_df['new_add'].str.replace('(','').str.replace(')','').str.replace('.','')
agoda_df['new_add'].replace(' ','', inplace=True, regex=True)
booking_df['new_add'].replace(' ','', inplace=True, regex=True)
traveloka_df['new_add'].replace(' ','', inplace=True, regex=True)
mytour_df['new_add'].replace(' ','', inplace=True, regex=True)

In [52]:
agoda_df['new_add']

0                                              02nguyenthiminhkhailoctho
1                                                       87bachdangtanlap
2                                             33atohienthanhtanlaptanlap
3                                        02nguyenthiminhkhaiparamaloctho
4                                                     32-34tranphuloctho
5                                                04hungvuongloctholoctho
6                                           39-41nguyenthiminhkhaitanlap
7                                               99nguyenthienthuatloctho
8                                                   plotd12abccamhaidong
9                                                 vinhninhvanvinhninhvan
10                                                       50tranphuloctho
11                                                    78-80tranphuloctho
12                                                    26-28tranphuloctho
13                                                 

In [53]:
#Check kết quả
for i in range(100):
    print(traveloka_df['Address'][i]+' ==> '+traveloka_df['new_add'][i])
print("\n====================\n")
for i in range(100):
    print(booking_df['Address'][i]+' ==> '+booking_df['new_add'][i])
print("\n====================\n")
for i in range(100):
    print(agoda_df['address'][i]+' ==> '+agoda_df['new_add'][i])
print("\n====================\n")
for i in range(100):
    print(mytour_df['Address'][i]+' ==> '+mytour_df['new_add'][i])

72-74 Hàng Buồm, Hàng Buồm, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000 ==> 72-74hangbuomhangbuomquanhoankiem
49 Lãn Ông, Hàng Bồ, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 108313 ==> 49lanonghangboquanhoankiem
11 Ngõ Trung Yên, phố Đinh Liệt, Hoan Kiếm, Hàng Bạc, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 106876 ==> 11trungyendinhliethoankiemhangbacquanhoankiem
Số 20, Tống Duy Tân, Phố Hàng Bống, Hàng Bông, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000 ==> 20tongduytanhangbonghangbongquanhoankiem
No 5, alley 148 Tran Duy Hung street, Yen Hoa ward, Cau Giay district, Hanoi, Phường Trung Hòa, Quận Cầu Giấy, Hà Nội, Việt Nam, 100000 ==> 5148tranduyhungyenhoacaugiayhaitrunghoaquancaugiay
235 Phố Huế, Phố Huế, Quận Hai Bà Trưng, Hà Nội, Việt Nam, 100000 ==> 235phohuehuequanhaibatrung
117 Trần Duy Hưng, Phường Trung Hòa, Quận Cầu Giấy, Hà Nội, Việt Nam, 100000 ==> 117tranduyhungtrunghoaquancaugiay
30 Phố Võng Thị , Bưởi, Tây Hồ, Hà Nội, Việt Nam, 100000 ==> 30vongthibuoitayho
Số 2 , Cầu Gỗ , Hoàn Kiếm , 

**Lưu kết quả xử lý tạm thời**

In [54]:
booking_s_tmp = "/content/drive/MyDrive/20212/THDL/data/temp_csv/booking_tmp.csv"
traveloka_s_tmp = "/content/drive/MyDrive/20212/THDL/data/temp_csv/traveloka_tmp.csv"
mytour_s_tmp = "/content/drive/MyDrive/20212/THDL/data/temp_csv/mytour_tmp.csv"
agoda_s_tmp = "/content/drive/MyDrive/20212/THDL/data/temp_csv/agoda_tmp.csv"

In [55]:
# agoda_df.to_csv(agoda_s_tmp, index=False, encoding='utf8')
# booking_df.to_csv(booking_s_tmp, index=False, encoding='utf8')
# traveloka_df.to_csv(traveloka_s_tmp, index=False, encoding='utf8')
# mytour_df.to_csv(mytour_s_tmp, index=False, encoding='utf8')

# **Đối sánh lược đồ**

*   **Chọn bộ dữ liệu chuẩn: booking.csv**


*   **Dựa trên tên các trường thuộc tính**

Sử dụng khoảng cách Levenshtein để tính độ tương đồng giữa tên của các trường dữ liệu


In [56]:
# #Đọc dữ liệu tmp
# agoda_tmp_df = pd.read_csv(agoda_s_tmp)
# booking_tmp_df = pd.read_csv(booking_s_tmp)
# mytour_tmp_df = pd.read_csv(mytour_s_tmp)
# traveloka_tmp_df = pd.read_csv(traveloka_s_tmp)

In [57]:
#Lấy tên các trường
agoda_ls = agoda_df.columns.values
print("Agoda: ")
print(agoda_ls)
booking_ls = booking_df.columns.values
print("Booking: ")
print(booking_ls)
mytour_ls = mytour_df.columns.values
print("Mytour: ")
print(mytour_ls)
traveloka_ls = traveloka_df.columns.values
print("Traveloka: ")
print(traveloka_ls)

Agoda: 
['city' 'hotel name' 'url' 'address' 'stars' 'price' 'rating' 'n_reviews'
 'image' 'reviews' 'facilities' 'nearby places' 'new_name' 'new_add']
Booking: 
['City' 'Hotel name' 'Url' 'Address' 'Stars' 'Price' 'Rating'
 'Number of reviews' 'Reviews' 'Facilities' 'Description' 'Nearby places'
 'Image' 'new_name' 'new_add']
Mytour: 
['City' 'Hotel name' 'Image' 'Url' 'Address' 'Stars' 'Min Price' 'Rating'
 'Number of reviews' 'Reviews' 'Facilities' 'Description' 'Nearby places'
 'new_name' 'new_add']
Traveloka: 
['City' 'Hotel name' 'Image' 'Url' 'Address' 'Stars' 'Min Price' 'Rating'
 'Number of reviews' 'Reviews' 'Facilities' 'Description' 'Nearby places'
 'new_name' 'new_add']


In [58]:
# def levenshtein(s, t):

#     rows = len(s)+1
#     cols = len(t)+1
#     distance = np.zeros((rows,cols),dtype = int)

    
#     for i in range(1, rows):
#         for k in range(1,cols):
#             distance[i][0] = i
#             distance[0][k] = k

      
#     for col in range(1, cols):
#         for row in range(1, rows):
#             if s[row-1] == t[col-1]:
#                 cost = 0
#             else:

#                 cost = 1
#             distance[row][col] = min(distance[row-1][col] + 1,      
#                                  distance[row][col-1] + 1,         
#                                  distance[row-1][col-1] + cost) 
#     return distance[row][col]

In [59]:
# levenshtein("tung", "tichhopdulieu")

In [60]:
#In ma trận khoảng cách của levenshtein
def printDistancesLS(distances, token1Length, token2Length):
    for t1 in range(token1Length + 1):
        for t2 in range(token2Length + 1):
            print(int(distances[t1][t2]), end=" ")
        print()

In [61]:
#Hàm tính độ đo levenshtein
def levenshteinDistance(str_1, str_2):
  #Tạo ma trận khoảng cách distances: str_1 ~ rows, str_2 ~ columns
  distances = np.zeros((len(str_1) + 1, len(str_2) + 1))

  #Khởi tạo hàng và cột đầu tiên của ma trận distances bằng các số nguyên bắt đầu từ 0
  for s1 in range(len(str_1) + 1):
    distances[s1][0] = s1
  for s2 in range(len(str_2) + 1):
    distances[0][s2] = s2

  # #In ma trận khởi tạo
  # printDistancesLS(distances, len(str_1), len(str_2))

  #Tính toán khoảng cách của tất cả các tiền tố
  #VD: với từ "hello" --> tiền tố: "h", "he", "hel", "hell", "hello"
  for s1 in range(1, len(str_1) + 1):
    for s2 in range(1, len(str_2) + 1):
      #Nếu 2 tiền tố giống nhau, thì khoảng cách ở ô hiện tại bằng giá trị ô trên cùng bên trái của ma trận 2x2 tương ứng
      if (str_1[s1-1] == str_2[s2-1]):
        distances[s1][s2] = distances[s1 - 1][s2 - 1]
      #Ngược lại, khoảng cách bằng giá trị min của 3 ô và cộng thêm 1
      else:
        dis = min(distances[s1][s2-1], distances[s1-1][s2], distances[s1-1][s2-1])
        distances[s1][s2] = dis + 1

  # # In ma trận kết quả
  # printDistancesLS(distances, len(str_1), len(str_2))

  #Kết quả là ô cuối cùng góc phải của ma trận
  return int(distances[len(str_1)][len(str_2)])
          

In [62]:
# #Test
# dis = levenshteinDistance("tung", "tichhopdulieu")
# dis

In [63]:
#Tìm min value của dictionary
def minimums(some_dict):
  positions = [] 
  min_value = float("inf")
  for k, v in some_dict.items():
    if v == min_value:
      positions.append((k, v))
    if v < min_value:
      min_value = v
      positions = []
      positions.append((k, v))

  return positions

In [64]:
# d = {320:1, 321:0, 322:3, 333:0}
# minimums(d)

In [65]:
def matchingName(name_ls, name):
  result = {}
  for i in booking_ls:
    x = {}
    for j in name_ls:
        x[j]=levenshteinDistance(j,i)
    x = {k: v for k, v in sorted(x.items(), key=operator.itemgetter(1))}
    result[i] = x

  print("Kết quả đối sánh dựa trên tên thuộc tính của bộ " + name + " với bộ Booking:\n")
  for it in result.keys():
    print(it + ': ', end=' ')
    print(minimums(result[it]))
  print("\n========================\n")

In [66]:
matchingName(agoda_ls, "Agoda")
matchingName(mytour_ls, "Mytour")
matchingName(traveloka_df, "Traveloka")

Kết quả đối sánh dựa trên tên thuộc tính của bộ Agoda với bộ Booking:

City:  [('city', 1)]
Hotel name:  [('hotel name', 1)]
Url:  [('url', 1)]
Address:  [('address', 1)]
Stars:  [('stars', 1)]
Price:  [('price', 1)]
Rating:  [('rating', 1)]
Number of reviews:  [('n_reviews', 10), ('reviews', 10)]
Reviews:  [('reviews', 1)]
Facilities:  [('facilities', 1)]
Description:  [('city', 8), ('rating', 8), ('facilities', 8)]
Nearby places:  [('nearby places', 1)]
Image:  [('image', 1)]
new_name:  [('new_name', 0)]
new_add:  [('new_add', 0)]


Kết quả đối sánh dựa trên tên thuộc tính của bộ Mytour với bộ Booking:

City:  [('City', 0)]
Hotel name:  [('Hotel name', 0)]
Url:  [('Url', 0)]
Address:  [('Address', 0)]
Stars:  [('Stars', 0)]
Price:  [('City', 4), ('Image', 4), ('Url', 4), ('Min Price', 4)]
Rating:  [('Rating', 0)]
Number of reviews:  [('Number of reviews', 0)]
Reviews:  [('Reviews', 0)]
Facilities:  [('Facilities', 0)]
Description:  [('Description', 0)]
Nearby places:  [('Nearby place

*   **Dựa trên bản ghi (Luật cú pháp)**

Dựa vào bộ dữ liệu Booking, nhóm có tập luật để đối sánh giá trị của từng bản ghi:


1.   "Url": phải chứa tên của domain.com tương ứng  // VD: booking phải chứa booking.com
2.   "Image": phải chứa ".jpg" hoặc ".png" hoặc ".jpeg" 

1.   "Stars" / "Rating": thuộc khoảng tương ứng
2.   "City": chỉ gồm một trong 8 giá trị "Hà Nội", "Hồ Chí Minh", "Đà Nẵng", "Đà Lạt", "Hạ Long", "Vũng Tàu","Nha Trang" và "Phú Quốc" 

1.   "Address": sau khi bắt được city thì address là trường có tỉ lệ chứa 1 trong 8 giá trị thành phố lớn nhất
2.   "Hotel name": riêng đối với trường này không có quy luật chung,
hơn nữa nhóm đã bắt được url ở phần trên do vậy đưa về chữ thường không dấu và dùng luật: tỉ lệ chứa từ "hotel" , "khach san", "chung cu", "can ho", "apartment", "hostel", "nghỉ dưỡng", "nghỉ mát", "resort", "spa", "home", "house", "homestay", "biệt thự", "villa", "du thuyền", "cruise", "bungalow", "restaurant", "residences", "studio", "bar" cao nhất











In [123]:
#Luật 1
def Rule1_Url(datadf, name):
  count = 0
  name = name.lower()
  print(name + ':')

  if name != "agoda":
    if name == "mytour":
      s = "mytour.vn"
    elif name == "traveloka":
      s = "traveloka.com"
    else:
      s = "booking.com"

    for i in datadf['Url']:
      if re.search(s, i):
        count += 1

    res = count/len(datadf['Url'])*100
    print(str(res) + "%")
  else:
    s = "agoda.com"
    for i in datadf['url']:
      if re.search(s, i):
        count += 1

    res = count/len(datadf['url'])*100
    print(str(res) + "%")
  print('-------------\n')

In [124]:
Rule1_Url(booking_df, "Booking")
Rule1_Url(agoda_df, "Agoda")
Rule1_Url(mytour_df, "Mytour")
Rule1_Url(traveloka_df, "Traveloka")

booking:
99.97936016511868%
-------------

agoda:
100.0%
-------------

mytour:
100.0%
-------------

traveloka:
100.0%
-------------



In [131]:
#Luật 2:
image_str = [".jpg", ".png", ".jpeg" ]
image_temp = '(?:% s)' % '|'.join(image_str)

# traveloka_i_s = ["ik.imagekit.io", "apr-asset"]
# traveloka_tmp = '(?:% s)' % '|'.join(traveloka_i_s)
# mytour_i_s = ["img.tripi.vn", "tripi-assets"]
# mytour_tmp = '(?:% s)' % '|'.join(mytour_i_s)
# agoda_tmp = 'agoda.net/hotelImages'
# booking_tmp = 'cf.bstatic.com/xdata/images'

# def Rule2_Image(datadf, name):
#   count = 0
#   if name != "Agoda":
#     if name == "Mytour":
#       for i in datadf['Image']:
#         if (re.search(image_temp,i)) & (re.search(mytour_tmp, i)):
#           count += 1
#     elif name == "Traveloka":
#       for i in datadf['Image']:
#         if (re.search(image_temp,i)) & (re.search(traveloka_tmp, i)):
#           count += 1
#     else:
#       for i in datadf['Image']:
#         if (re.search(image_temp,i)) & (re.search(booking_tmp, i)):
#           count += 1

#     res = count/len(datadf['Image'])*100
#     print(str(res) + "%")
#   else:
#     for i in datadf['image']:
#       if (re.search(image_temp,i)) & (re.search(agoda_tmp, i)):
#         count += 1

#     res = count/len(datadf['url'])*100
#     print(str(res) + "%")

def Rule2_Image(datadf, name):
  count = 0
  name = name.lower()
  print(name + ':')

  if name != "agoda":
    for i in datadf['Image']:
      if re.search(image_temp,i):
        count += 1

    res = count/len(datadf['Image'])*100
    print(str(res) + "%")
  else:
    for i in datadf['image']:
      if re.search(image_temp,i):
        count += 1

    res = count/len(datadf['image'])*100
    print(str(res) + "%")
  print('-------------\n')

In [132]:
Rule2_Image(booking_df, "Booking")
Rule2_Image(agoda_df, "Agoda")
Rule2_Image(mytour_df, "Mytour")
Rule2_Image(traveloka_df, "Traveloka")

booking:
99.97936016511868%
-------------

agoda:
100.0%
-------------

mytour:
98.54932301740811%
-------------

traveloka:
100.0%
-------------



In [129]:
#Luật 3:
def Rule3_StarRating(datadf, name):
  count_s = 0
  count_r = 0
  name = name.lower()
  print(name + ':')

  if name != "agoda":
    for i in datadf['Stars']:
      if i <= 5.0:
        count_s += 1
    for j in datadf['Rating']:
      if j <= 10.0:
        count_r += 1

    res_s = count_s/len(datadf['Stars'])*100
    print("Star: "+str(res_s) + "%")
    res_r = count_r/len(datadf['Rating'])*100
    print("Rating: " + str(res_r) + "%")
  else:
    for i in datadf['stars']:
      if i/10 <= 5.0:
        count_s += 1
    for j in datadf['rating']:
      if j <= 10.0:
        count_r += 1

    res_s = count_s/len(datadf['stars'])*100
    print("Star: "+str(res_s) + "%")
    res_r = count_r/len(datadf['rating'])*100
    print("Rating: " + str(res_r) + "%")
  print('-------------\n')

In [130]:
Rule3_StarRating(booking_df, "Booking")
Rule3_StarRating(agoda_df, "Agoda")
Rule3_StarRating(mytour_df, "Mytour")
Rule3_StarRating(traveloka_df, "Traveloka")

booking:
Star: 100.0%
Rating: 100.0%
-------------

agoda:
Star: 100.0%
Rating: 100.0%
-------------

mytour:
Star: 100.0%
Rating: 100.0%
-------------

traveloka:
Star: 100.0%
Rating: 100.0%
-------------



In [143]:
#Luật 4-5:
city_str = ["Hà Nội", "Hồ Chí Minh", "Đà Nẵng", "Đà Lạt", "Quảng Ninh" 
            "Hạ Long", "Vũng Tàu", "Nha Trang", "Phú Quốc",
            "Hà Nội", "Hạ Long", "Vũng Tàu", "Phú Quốc", "Quảng Ninh"
            "Nha Trang", "Đà Nẵng", "Đà Lạt", "Hồ Chí Minh"]
city_temp = '(?:% s)' % '|'.join(city_str)

def Rule45_CityAddress(datadf, name):
  count_c = 0
  count_a = 0
  name = name.lower()
  print(name + ':')

  if name != "agoda":
    for i in datadf['City']:
      if re.search(city_temp,i):
        count_c += 1
    for j in datadf['Address']:
      if re.search(city_temp,j):
        count_a += 1

    res_c = count_c/len(datadf['City'])*100
    res_a = count_a/len(datadf['Address'])*100
    print("City: " + str(res_c) + "%")
    print("Address: " + str(res_a) + "%")
  else:
    for i in datadf['city']:
      if re.search(city_temp,i):
        count_c += 1
    for j in datadf['address']:
      if re.search(city_temp,j):
        count_a += 1

    res_c = count_c/len(datadf['city'])*100
    res_a = count_a/len(datadf['address'])*100
    print("City: " + str(res_c) + "%")
    print("Address: " + str(res_a) + "%")
  print('-------------\n')

In [144]:
Rule45_CityAddress(booking_df, "Booking")
Rule45_CityAddress(agoda_df, "Agoda")
Rule45_CityAddress(mytour_df, "Mytour")
Rule45_CityAddress(traveloka_df, "Traveloka")

booking:
City: 99.97936016511868%
Address: 99.97936016511868%
-------------

agoda:
City: 93.06071871127634%
Address: 93.55638166047088%
-------------

mytour:
City: 97.96905222437138%
Address: 97.87234042553192%
-------------

traveloka:
City: 96.41170915958452%
Address: 96.41170915958452%
-------------



In [149]:
#Luật 6:
hotel_str = ["hotel" , "khách sạn", "chung cư", "căn hộ", "apartment", "hostel",
             "nghỉ dưỡng", "nghỉ mát", "resort", "spa", "home", "house", "homestay", 
             "biệt thự", "villa", "du thuyền", "cruise", "bungalow", "restaurant", 
             "residences", "studio", "bar", "nhà"]
hotel_temp = '(?:% s)' % '|'.join(hotel_str)

def Rule6_Hotelname(datadf, name):
  count = 0
  name = name.lower()
  print(name + ':')

  if name != "agoda":
    for i in datadf['Hotel name']:
      i = i.lower()
      if re.search(hotel_temp,i):
        count += 1

    res = count/len(datadf['Hotel name'])*100
    print(str(res) + "%")
  else:
    for i in datadf['hotel name']:
      i = i.lower()
      if re.search(hotel_temp,i):
        count += 1

    res = count/len(datadf['hotel name'])*100
    print(str(res) + "%")
  print('-------------\n')

In [150]:
Rule6_Hotelname(booking_df, "Booking")
Rule6_Hotelname(agoda_df, "Agoda")
Rule6_Hotelname(mytour_df, "Mytour")
Rule6_Hotelname(traveloka_df, "Traveloka")

booking:
99.93808049535605%
-------------

agoda:
93.3085501858736%
-------------

mytour:
91.68278529980658%
-------------

traveloka:
88.47969782813976%
-------------



In [151]:
#Check cho bản ghi cột khác nhau
#Ví dụ: bản ghi cột address --> Độ tương tự rất thấp
count=0
for i in booking_df['Address']:
  i=unidecode.unidecode(str.lower(i))
  if re.search(hotel_temp,i):
      count+=1
print(count/len(booking_df['Address'])*100,end=' ')
print('%')

3.281733746130031 %
